# Web Spider Tutorial

## 1. 网络爬虫简介

**网络爬虫**指通过**脚本**自动从网页获取**数据**（文本、图像、音频、视频），并保存至本地的过程。例如：
+ 获取人民网近三个月的全部新闻
+ 给定关键词，从百度图片获取100张和关键词有关的图片
+ 获取给定BV号的B站视频

## 2. 基于`requests`和`BeautifulSoup`的Python网络爬虫

基于Python的网络爬虫一般可以通过两个模块实现：`requests`和`BeautifulSoup`。
+ `requests`：用于向url**发出请求**，**获取响应**
+ `BeautifulSoup`：用于**解析**通过`requests`获得的响应，**获取所需数据**。其有时会配合正则表达式模块`re`一起使用。

## 3. 本教程项目介绍

本教程通过B站爬虫项目介绍爬虫的相关概念和基本流程。
+ 输入：一个BV号，如`BV19B4y1W76i`
+ 输出：视频基本信息（标题、时间、简介、标签、分p信息）和视频内容（mp4格式。如果视频有分p，获取每一p的内容）。

接下来，我们以获取BV号为`BV19B4y1W76i`的视频的基本信息和内容为例。

### 3.1 B站视频BV号

首先，介绍B站视频BV号（即视频ID）的获取方法。

1. 打开某个B站的视频。
2. 获取其url。url可能是这样的：
    + https://www.bilibili.com/video/BV19B4y1W76i?spm_id_from=333.337.search-card.all.click&vd_source=cb6bdc56db66ac895c0f3d6912c94028
3. 红色字体部分即为BV号。<font color=gray><i><u>另外，BV号后面的参数信息（即?及其后面的内容）的有无不影响对该url的访问。</u></i></font>
    + https:</div>//www</div>.bilibili.com/video/<font color=red>BV19B4y1W76i</font>?spm_id_from=333.337.search-card.all.click&vd_source=cb6bdc56db66ac895c0f3d6912c94028

相反，给定视频BV号（如`BV19B4y1W76i`），我们也可以构造这个视频的url。在BV号前面加上`https://www.bilibili.com/video/`即可得到该BV号对应的url，即：
+ https://www.bilibili.com/video/BV19B4y1W76i

### 3.2 通过BV号构建视频网址的url

基于上述介绍，我们可以写一个函数`construct_url`，接收BV号，返回视频的url。

In [1]:
def construct_url(bv_id):
    """Construct a url with the given bv id.

    :param bv_id: BV id.
    :return: Constructed url.
    """

    return f"https://www.bilibili.com/video/{bv_id}"

Python基础回顾：
+ 函数构建
+ 函数文档
+ 字符串格式化

现在我们来测试一下这个函数。将BV号`BV19B4y1W76i`作为传入函数，获取构造的url。

In [2]:
bv_id = "BV19B4y1W76i"
url = construct_url(bv_id=bv_id)

print(url)

https://www.bilibili.com/video/BV19B4y1W76i


Python基础回顾：
+ 函数调用
+ 输出语句

## 4. 使用`requests`模块请求网页内容

爬虫的第一步是**向网页的url发出请求，并获取响应（即网页内容，一般为html）**，我们使用`requests`模块完成这个任务。

我们需要获取的内容可以通过浏览器直观看到，步骤如下：
1. 通过url访问网页，如：https://www.bilibili.com/video/BV19B4y1W76i
2. 打开网页源代码窗口：鼠标右键，点击“查看网页源代码”（取决于浏览器，Chrome的叫法是这个。有些浏览器是“审查元素”）。或者也可以通过`F12`打开。打开的页面如下。左侧红框部分就是我们需要通过`requests`获取的内容，即网页的html。（注意，需要在顶部导航栏选择“Elements”（不同浏览器可能叫法不同）才能看到。不过一般默认显示的就是“Elements”的内容）

<div align="center"><img src="f12.png" width="600" align="center" /></div>

### 4.1 主流程

这一节介绍使用`requests`模块获取请求网页内容的主流程。分为两步：
1. 向url发出请求，获取响应内容
2. 从响应内容获取html

首先，我们导入`requests`模块。

In [3]:
import requests

Python基础回顾：
+ 模块导入

#### 4.1.1 向url发出请求，获取响应内容

导入模块后，使用`requests`模块的`get`方法向刚刚构建的`url`发出请求，得到响应内容`r`。

In [4]:
r = requests.get(url=url)

Python基础回顾：
+ 类方法的调用

我们可以通过输出`r`查看请求是否成功。如果请求成功，将输出`<Response [200]>`

In [5]:
print(r)

<Response [200]>


#### 4.1.2 从响应内容获取html

获取响应内容`r`之后，我们通过`r`的`text`属性获取网页的html。

In [33]:
html = r.text

# 输出html的前1000个字符
print(html[:1000])

<!DOCTYPE html><html lang="zh-Hans"><head itemprop="video" itemscope itemtype="http://schema.org/VideoObject"><meta name="format-detection" content="telephone=no, email=no"><meta http-equiv="Content-Type" content="text/html" charset="utf-8"><meta name="spm_prefix" content="333.788"><meta name="referrer" content="no-referrer-when-downgrade"><meta name="applicable-device" content="pc"><meta http-equiv="Cache-Control" content="no-transform"><meta http-equiv="Cache-Control" content="no-siteapp"><link rel="stylesheet" href="//s1.hdslb.com/bfs/static/jinkela/long/laputa-css/map.css"><link rel="stylesheet" href="//s1.hdslb.com/bfs/static/jinkela/long/laputa-css/light_u.css"><link id="__css-map__" rel="stylesheet" href="//s1.hdslb.com/bfs/static/jinkela/long/laputa-css/light.css"><script type="text/javascript">window.webAbTest={"pc_player_autoplay_switch_reset":"1","buvidsplit":"7"}</script> <title data-vue-meta="true">[中英字幕]吴恩达2022机器学习 machine learning specialization_哔哩哔哩_bilibili</title> <me

#### 4.1.3 主流程核心代码

基于此，主流程的核心代码如下所示。

In [7]:
import requests

r = requests.get(url=url)
html = r.text

### 4.3 处理网络请求的异常

网络请求可能由于网络条件和服务器响应意愿等原因产生各种各样的异常，如请求超时、拒绝访问、不可达等问题。为了处理这些异常，通常将代码写成如下形式。主要有两个改动：
1. 调用`get`方法后，通过`r.raise_for_status()`确认请求是否成功。如果请求不成功，这行代码会抛出异常。
2. 通过异常处理`try-except`处理`raise_for_status`方法可能抛出的异常。

In [8]:
html = ""
try:
    # Send the request.
    r = requests.get(url=url)
    # Raise an exception if something goes wrong.
    r.raise_for_status()
    
    html = r.text
except:
    html = ""

Python基础回顾：
+ 异常处理

### 4.4 封装成函数

为了更好地模块化，现在我们将上面的请求网页内容的代码封装成函数`request`。

In [9]:
def request(url):
    """Send a request.

    :param url: The url to which the request is sent.
    :return: Response from the server of the url.
    """

    try:
        # Send the request.
        r = requests.get(url=url)
        # Raise an exception if something goes wrong.
        r.raise_for_status()

        return r
    except:
        return None

Python基础回顾：
+ 空值

需要注意，该函数返回的是请求成功后的响应对象`r`。不直接返回网页的html（即`r.text`）的原因是我们有时候请求的是音频和视频等二进制资源，而不是html。在请求二进制资源的时候，我们获取资源使用的是`r.content`。

因此，我们通过以下代码获取请求的html。

In [10]:
html = request(url=url).text

## 5. 使用`BeautifulSoup`模块解析网页并获取所需内容

使用`requests`模块请求得到网页内容后，我们使用`BeautifulSoup`模块获取所需的内容，如文本、图片url、视频url等。

### 5.1 主流程

这一节介绍使用`BeautifulSoup`解析并获取数据的主流程。包括：
1. 解析html
2. 通过浏览器定位所需数据对应的标签
3. 在soup中获取标签对象
4. 从标签对象中获取所需数据

我们以获取视频的标题为例。上述BV号`BV19B4y1W76i`对应的视频标题为：
+ [中英字幕]吴恩达2022机器学习 machine learning specialization。

我们接下来从获得的`html`中获取该标题。

#### 5.1.1 解析html

首先，导入该模块，并使用该模块解析通过`requests`得到的html。

In [11]:
from bs4 import BeautifulSoup

# Parse the html.
soup = BeautifulSoup(
    markup=html,
    features="html.parser"
)

Python基础回顾：
+ 模块导入（2）

#### 5.1.2 通过浏览器定位所需数据对应的标签

接下来，我们需要通过浏览器定位标题所在的标签：
1. 打开网页源代码窗口（右键后点击“查看网页源代码”/“审查元素”，或者`F12`。）
2. 点击左上角按钮
3. 点击按钮后，在网页中点击视频标题，可以看到源代码窗口会定位到标题所在的标签位置。
<div align="center"><img src="locate_title.png" width="800" align="center" /></div>

可以看到，定位到的标签是：
```html
<h1 title="[中英字幕]吴恩达2022机器学习 machine learning specialization" class="video-title tit">[中英字幕]吴恩达2022机器学习 machine learning specialization</h1>
```

#### 5.1.3 在soup中获取标签对象

接下来，我们通过`find`方法从`soup`中获取这个标签对应的标签对象。`find`方法的参数`name`需要传入标签的类型。上述标签的类型是`h1`，因此传入`"h1"`。

In [12]:
title_h1 = soup.find(name="h1")

# 看看获取到的标签对象。
print(f"获取到的标签对象：{title_h1}")
# 查看其类型。
print(f"该标签对象的类型是：{type(title_h1)}")

获取到的标签对象：<h1 class="video-title tit" title="[中英字幕]吴恩达2022机器学习 machine learning specialization">[中英字幕]吴恩达2022机器学习 machine learning specialization</h1>
该标签对象的类型是：<class 'bs4.element.Tag'>


Python基础回顾：
+ 查看变量类型

#### 5.1.4 从标签对象中获取所需数据

接下来，我们从`title_h1`中获取标题。从上述标签可以看到，标题被两个`h1`标签夹起来，形式为`<h1>标题</h1>`。我们通过`text`属性即可访问这些被标签夹起来的内容，即此处的标题。

In [13]:
title = title_h1.text

print(title)

[中英字幕]吴恩达2022机器学习 machine learning specialization


#### 5.1.5 主流程核心代码

基于此，主流程的核心代码如下所示。

In [14]:
from bs4 import BeautifulSoup

# Parse the html.
soup = BeautifulSoup(
    markup=html,
    features="html.parser"
)

# Get title.
title_h1 = soup.find(name="h1")  # 通过“5.1.2 通过浏览器定位所需数据对应的标签”得知这个标签是一个h1标签。
title = title_h1.text  # 通过“5.1.2 通过浏览器定位所需数据对应的标签”得知标题被两个标签（一开一闭）夹起来，因此使用.text。

### 5.2 一些类型的标签可能有很多个！

现在，我们尝试获取视频的时间，步骤类似。通过浏览器，我们可以看到视频时间的标签是一个`span`标签。（<font color="red">定位后需要点击▶展开`span`标签才能看到日期</font>）

<div align="center"><img src="locate_date.png" width="800" align="center" />

我们尝试获取日期的`span`标签对象。

In [15]:
date_span = soup.find(name="span")

# 输出看看
print(date_span)

<span class="van-dialog__title">确认公开发布笔记？</span>


Ohhhhhhhhhhh。好像不是我们想要的。原因是，有很多很多个`span`标签（在网页源码`Ctrl F`搜索一下`<span>`就知道了）。`find`方法不知道你想要哪一个。

<div align="center"><img src="many_spans.png" width="800" align="center" />

因此，我们需要更精确的查找。我们再看看包着视频日期的`span`标签：

```html
<span class="pudate item">
    <svg t="1642588113899" viewBox="0 0 1024 1024" version="1.1" xmlns="http://www.w3.org/2000/svg" p-id="6827" xmlns:xlink="http://www.w3.org/1999/xlink" width="200" height="200" class="icon">
        <defs></defs>
        <path d="M167.024 512a344.976 344.976 0 1 1 689.952 0 344.976 344.976 0 0 1-690 0zM512 106.976a405.024 405.024 0 1 0 0 810.048 405.024 405.024 0 0 0 0-810z m30 235.008a30 30 0 1 0-60 0V512c0 7.968 3.168 15.6 8.784 21.216l120 120a30 30 0 1 0 42.432-42.432L542 499.52V341.984z" fill="#9499A0" p-id="6828"></path>
    </svg>

      2022-06-16 09:38:53
</span>
```

这个`span`标签有一个`class`属性，这个属性有两个值：`pudate`和`item`。通过搜索发现，`class`属性的这两个值中`pudate`只出现了一次，因此我们可以用它完成更精确的定位。我们只需在`find`方法中指定`class`属性对应的值即可。需要注意，<font color="red"><b>由于标签的这个属性`class`和python的类定义关键字`class`冲突了，因此我们实际要写的是`class_`。</b></font>

由于视频日期也被两个`span`标签夹着，即`<span>视频日期</span>`的形式，因此，也使用`text`获取。你可能会问，`span`标签里面还有一个`svg`标签，不会有影响吗？？其实是不会的。实际上，<font color="red"><b>在很多时候，我们获取标签对象之后都可以先尝试使用`text`获取内容，一般不会有问题。有问题再想办法~</b></font>

In [16]:
date_span = soup.find(
    name="span",
    class_="pudate"
)
date = date_span.text

print(date)



      2022-06-16 09:38:53


很明显，我们还需要使用字符串方法`strip`去掉多余的空格。

In [17]:
date = date.strip()

print(date)

2022-06-16 09:38:53


<!-- ### 5.3 标签套娃以及`find_all` -->

<!-- 有时候，我们使用`find`获取标签对象后，还希望从这个标签对象中获取其包含的某些标签对象，此时，对获取到的标签对象继续`find`即可。

还有时候，我们希望获取的是一系列的标签对象，如一个列表的每一行，这时候，需要使用`find_all`。

下面，通过获取视频的分p信息介绍这两种情况的应对方法。 -->

## 6. `find_all`

有时候，我们希望获取一系列数据，如表格每一行。这时候，我们需要使用`find_all`。假如我们需要获取视频的全部tag，如下图。可以看到，每个tag都被一个`li`标签包着，而这一系列`li`标签被一个`ul`标签包着。

<div align="center"><img src="locate_tags.png" width="800" align="center" />

首先，我们定位这个`ul`标签。

In [23]:
tags_ul = soup.find(
    name="ul",
    class_="tag-area"
)

接下来，我们需要获取这个`ul`里面的所有`li`，分别从这些`li`里面获取tag。为此，我们需要使用`find_all`。其使用方法和`find`类似。<font color=gray><i><u>（实际上调用`find`之后内部也会调用`find_all`，所以`find`实际上是`find_all`只找一个标签的特殊形式。）</u></i></font>

In [19]:
tags = []
for tag_li in tags_ul.find_all(name="li"):
    tag = tag_li.text.strip()
    tags.append(tag)
        
# 输出看看。
print(tags)

['编程', '科学', '知识', '科学科普', 'AI', '机器学习', '吴恩达', '公开课创作激励x新星计划', '']


Python基础回顾：
+ 列表
+ for循环
+ while循环

基本没问题，不过`tags`列表最后有一个空串。这是因为，实际上找到的有些`li`标签不是tag，`strip`之后会变成空串（长度为0）。因此，我们需要做个过滤。

In [24]:
tags = []
for tag_li in tags_ul.find_all(name="li"):
    tag = tag_li.text.strip()
    
    # 实际上找到的有些li标签不是tag，strip之后会变成空串（长度为0）。因此做个过滤。
    if len(tag) != 0:
        tags.append(tag)
        
# 输出看看。
print(tags)

['编程', '科学', '知识', '科学科普', 'AI', '机器学习', '吴恩达', '公开课创作激励x新星计划']


Pytohn基础回顾：
+ if条件语句
+ 字符串长度

## 7. 网页内容的差异

有时候，通过浏览器看到的网页源码和使用`requests`得到的会有差异。

举个例子，我们使用前面介绍的方法获取视频分p信息。
<div align="center"><img src="locate_pages.png" width="800" align="center" />

直接上代码~

In [36]:
print_count = 10  # 这是为了不要输出太多。

for li_p in soup.find(name="ul", class_="list-box").find_all("li"):
    print(li_p)
    
    print_count -= 1
    if print_count == 0:
        break  # 输出10个看看就行了。剩下的无视掉。

<li><!-- --></li>
<li><!-- --></li>
<li><!-- --></li>
<li><!-- --></li>
<li><!-- --></li>
<li><!-- --></li>
<li><!-- --></li>
<li><!-- --></li>
<li><!-- --></li>
<li><!-- --></li>


可以看到，啥也没有。这是因为，这个`ul`里面的东西是浏览器渲染出来的。我们代码中通过`requests`得到的`html`解析出来的`soup`没有经过浏览器渲染，因此拿不到这里的信息。

然而实际上，我们在`requests`得到的`html`里面直接搜索某一个p的标题，是能搜到的（直接在4.1.2节获取的`html`搜索就可以）。通过分析`requests`请求得到的`html`，我们发现分p信息藏在一个`script`标签里面（如下，仅展示一部分）。而这一部分信息通过浏览器是看不到的。因此，有时候通过浏览器看到的网页源码和使用`requests`得到的会有差异。如果我们通过浏览器定位得不到需要的东西，可以试试**直接在`requests`请求得到的`html`中通过搜索手动定位**。在这里，通过`html`中直接定位找到了所需的`script`标签。

```html
<script>window.__INITIAL_STATE__={"aid":600070218,"bvid":"BV19B4y1W76i","p":1,"episode":"","videoData":{"bvid":"BV19B4y1W76i","aid":600070218,"videos":100,"tid":201,"tname":"科学科普","copyright":2,"pic":"http:\u002F\u002Fi1.hdslb.com\u002Fbfs\u002Farchive\u002F73ce549a9b791241679533da1af72dad1ec8ed90.jpg","title":"[中英字幕]吴恩达2022机器学习 machine learning specialization","pubdate":1655343533,"ctime":1655342382,"desc":"吴恩达2022新版机器学习 machine learning specialization\r\n欢迎进群交流，所有资源也会放到群内：484266833\r\n课程官网：https:\u002F\u002Fwww.coursera.org\u002Fspecializations\u002Fmachine-learning-introduction\r\nGitHub：https:\u002F\u002Fgithub.com\u002Fkaieye\u002F2022-Machine-Learning-Specialization\r\n课程代码及测验内容已更新完毕\r\n欢迎pull request","desc_v2":[{"raw_text":"吴恩达2022新版机器学习 machine learning specialization\r\n欢迎进群交流，所有资源也会放到群内：484266833\r\n课程官网：https:\u002F\u002Fwww.coursera.org\u002Fspecializations\u002Fmachine-learning-introduction\r\nGitHub：https:\u002F\u002Fgithub.com\u002Fkaieye\u002F2022-Machine-Learning-Specialization\r\n课程代码及测验内容已更新完毕\r\n欢迎pull request","type":1,"biz_id":0}],"state":0,"duration":48921,"rights":{"bp":0,"elec":0,"download":1,"movie":0,"pay":0,"hd5":0,"no_reprint":0,"autoplay":1,"ugc_pay":0,"is_cooperation":0,"ugc_pay_preview":0,"no_background":0,"clean_mode":0,"is_stein_gate":0,"is_360":0,"no_share":0,"arc_pay":0,"free_watch":0},"owner":{"mid":295896691,"name":"渚隰","face":"http:\u002F\u002Fi1.hdslb.com\u002Fbfs\u002Fface\u002F894091fa465069c4690ca0452ec6d5beedc2db19.jpg"},"stat":{"aid":600070218,"view":59232,"danmaku":258,"reply":185,"favorite":6903,"coin":939,"share":400,"now_rank":0,"his_rank":0,"like":1761,"dislike":0,"evaluation":"","argue_msg":"","viewseo":59232},"dynamic":"","cid":747974480,"dimension":{"width":1280,"height":720,"rotate":0},"premiere":null,"teenage_mode":0,"is_chargeable_season":false,"no_cache":false,"pages":[{"cid":747974480,"page":1,"from":"vupload","part":"1.1 欢迎来到机器学习!","duration":165,"vid":"","weblink":"","dimension":{"width":1280,"height":720,"rotate":0},"first_frame":"http:\u002F\u002Fi1.hdslb.com\u002Fbfs\u002Fstoryff\u002Fn22061601rlyy7ii8t77b1zm614sjchw_firsti.jpg"},{"cid":747961770,"page":2,"from":"vupload","part":"1.2 机器学习的应用","duration":269,"vid":"","weblink":"","dimension":{"width":1280,"height":720,"rotate":0},"first_frame":"http:\u002F\u002Fi1.hdslb.com\u002Fbfs\u002Fstoryff\u002Fn220616153e4i8j9uhy6lx2c55ps5xkh_firsti.jpg"},{"cid":747961854,"page":3,"from":"vupload","part":"2.1 什么是机器学习","duration":336,"vid":"","weblink":"","dimension":{"width":1280,"height":720,"rotate":0},"first_frame":"http:\u002F\u002Fi1.hdslb.com\u002Fbfs\u002Fstoryff\u002Fn22061615ajo6u4qba5ah14jc7ky5n85_firsti.jpg"},{"cid":747961924,"page":4,"from":"vupload","part":"2.2 监督学习 part 1","duration":417,"vid":"","weblink":"","dimension":{"width":1280,"height":720,"rotate":0},"first_frame":"http:\u002F\u002Fi2.hdslb.com\u002Fbfs\u002Fstoryff\u002Fn220616081d3niogxxcusuq9eep0su5l_firsti.jpg"},{"cid":747964817,"page":5,"from":"vupload","part":"2.3 监督学习 part 2","duration":437,"vid":"","weblink":"","dimension":{"width":1280,"height":720,"rotate":0},"first_frame":"http:\u002F\u002Fi1.hdslb.com\u002Fbfs\u002Fstoryff\u002Fn22061608i0zfo2mr4uq52dwqny1v1ox_firsti.jpg"},

...
</script>
```

此后，获取分p信息就不难了。

In [43]:
import json
import re

window_initial_state_script = ""  # 存放上面的script。

for script in soup.find_all(name="script"):  # 有很多script标签。
    try:
        # 在这做做过滤。需要的script标签的内容包含"window.__INITIAL_STATE__"和"pages"。
        # 这里用.string而不是.text，因为.text得不到。
        # .string和.text是有区别的，不过不用想太多，.text不行就用.string。
        if "window.__INITIAL_STATE__" in script.string and "pages" in script.string:  
            window_initial_state_script = script.string
            break  # 拿到了就跑。
    except TypeError:  # 这里做异常处理是因为，对于find_all找出来的script，有些调用.string会出错。
        pass
    
# 使用正则表达式抽取script里面包含的json字符串。
pattern = re.compile(pattern=r"(\{.*\});")
window_initial_state_json = pattern.search(string=window_initial_state_script).group(1)

# json转dict。
window_initial_state_dict = json.loads(s=window_initial_state_json)

# 找出分p信息。
pages = window_initial_state_dict["videoData"]["pages"]
pages = {
    int(page_dict['page']): page_dict['part']
    for page_dict in pages
}

# 输出前10个看看。
print_count = 10
for page_id, page_name in pages.items():
    print(f"[p{page_id}]: {page_name}")
    
    print_count -= 1
    if print_count == 0:
        break

[p1]: 1.1 欢迎来到机器学习!
[p2]: 1.2 机器学习的应用
[p3]: 2.1 什么是机器学习
[p4]: 2.2 监督学习 part 1
[p5]: 2.3 监督学习 part 2
[p6]: 2.4 非监督学习 part 1
[p7]: 2.5 非监督学习 part 2
[p8]: 2.6 Jupyter Notebooks
[p9]: 3.1 线性回归模型 part 1
[p10]: 3.2 线性回归模型 part 2


Pytohn基础回顾：
+ *正则表达式
+ *json字符串转字典
+ 字典
+ 列表/元组/字典推导式
+ 类型转换

## 8. 总结：视频信息的获取

In [48]:
def get_info(url):
    """Get video info.

    :param url: Url of the video.
    :return: Video info, including title, date, introduction, tags and page list, wrapped into a dict.
    """

    # Obtain the html text of the url.
    html = request(url=url).text

    # Parse the html.
    soup = BeautifulSoup(
        markup=html,
        features="html.parser"
    )

    # Get the video title.
    title = soup.find(
        name="h1",
        class_="video-title"
    ).text.strip()

    # Get the video date.
    date = soup.find(
        name="span",
        class_="pudate"
    ).text.strip()

    # Get the video introduction.
    intro = soup.find(
        name="span",
        class_="desc-info-text"
    ).text.strip()

    # Get the video tags.
    tags = []
    for tag_li in soup.find(
            name="ul",
            class_="tag-area"
    ).find_all(name="li"):
        tag = tag_li.text.strip()
        # It's really a tag if it is not an empty string.
        if len(tag) != 0:
            tags.append(tag)

    # Get the video pages.
    window_initial_state_script = ""  # The script contains page info.
    for script in soup.find_all(name="script"):
        try:
            if "window.__INITIAL_STATE__" in script.string and "pages" in script.string:
                window_initial_state_script = script.string
                break
        except TypeError:
            pass
    # Extract the json from the script.
    pattern = re.compile(pattern=r"(\{.*\});")
    window_initial_state_json = pattern.search(string=window_initial_state_script).group(1)
    # Convert the json to dict.
    window_initial_state_dict = json.loads(s=window_initial_state_json)
    # Extract page info.
    pages = window_initial_state_dict["videoData"]["pages"]
    pages = {
        int(page_dict['page']): page_dict['part']
        for page_dict in pages
    }

    return {
        "title": title,
        "date": date,
        "intro": intro,
        "tags": tags,
        "pages": pages,
    }


bv_id = "BV19B4y1W76i"

url = construct_url(bv_id=bv_id)
info = get_info(url=url)

for k, v in info.items():
    print(f"[{k}]: {v}")

[title]: [中英字幕]吴恩达2022机器学习 machine learning specialization
[date]: 2022-06-16 09:38:53
[intro]: 吴恩达2022新版机器学习 machine learning specialization
欢迎进群交流，所有资源也会放到群内：484266833
课程官网：https://www.coursera.org/specializations/machine-learning-introduction
GitHub：https://github.com/kaieye/2022-Machine-Learning-Specialization
课程代码及测验内容已更新完毕
欢迎pull request
[tags]: ['编程', '科学', '知识', '科学科普', 'AI', '机器学习', '吴恩达', '公开课创作激励x新星计划']
[pages]: {1: '1.1 欢迎来到机器学习!', 2: '1.2 机器学习的应用', 3: '2.1 什么是机器学习', 4: '2.2 监督学习 part 1', 5: '2.3 监督学习 part 2', 6: '2.4 非监督学习 part 1', 7: '2.5 非监督学习 part 2', 8: '2.6 Jupyter Notebooks', 9: '3.1 线性回归模型 part 1', 10: '3.2 线性回归模型 part 2', 11: '3.3 代价函数', 12: '3.4 代价函数的直观理解', 13: '3.5 可视化代价函数', 14: '3.6 可视化的例子', 15: '4.1 梯度下降', 16: '4.2 实现梯度下降', 17: '4.3 梯度下降的直观理解', 18: '4.4 学习率', 19: '4.5 线性回归中的梯度下降', 20: '4.6 运行梯度下降', 21: '5.1 多类特征', 22: '5.2 向量化 part 1', 23: '5.3 向量化 part 2', 24: '5.4 多元线性回归的梯度下降法', 25: '6.1 特征缩放 part 1', 26: '6.2 特征缩放 part 2', 27: '6.3 检查梯度下降是否收敛', 28: '6.4 学习率的

## 9. 分p视频内容爬取

接下来，我们把每一p的视频爬下来，并以mp4形式保存。

首先讲点需要用到的东西。

### 9.1 每一P的URL

首先，点击第1p、第2p、第3p，看看url的格式。
+ 第1p：https://www.bilibili.com/video/BV19B4y1W76i?spm_id_from=333.337.search-card.all.click&vd_source=cb6bdc56db66ac895c0f3d6912c94028
+ 第2p：https://www.bilibili.com/video/BV19B4y1W76i?p=2&vd_source=cb6bdc56db66ac895c0f3d6912c94028
+ 第3p：https://www.bilibili.com/video/BV19B4y1W76i?p=3&vd_source=cb6bdc56db66ac895c0f3d6912c94028

够了够了。来找规律。先把没用的东西删掉。
+ 第1p：https://www.bilibili.com/video/BV19B4y1W76i
+ 第2p：https://www.bilibili.com/video/BV19B4y1W76i?p=2
+ 第3p：https://www.bilibili.com/video/BV19B4y1W76i?p=3

规律找到了，顺便帮第1p加点东西。
+ 第1p：https://www.bilibili.com/video/BV19B4y1W76i?p=1
+ 第2p：https://www.bilibili.com/video/BV19B4y1W76i?p=2
+ 第3p：https://www.bilibili.com/video/BV19B4y1W76i?p=3

然后就可以`for`遍历了~

### 9.2 请求头

使用`requests`请求的时候，有时候需要添加请求头（以字典的形式），给服务器提供更多信息。为了爬取视频内容，我们需要如下请求头。

```python
{
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36',
    'Referer': url,  # Necessary for requesting audio/video content.
}
```

其中，`'User-Agent'`是为了告诉服务器，发出网络请求的不是一个爬虫，而是浏览器。通过`requests`发出的请求中，其请求头的`'User-Agent'`和浏览器发出的不太一样，而有些服务器会通过`'User-Agent'`判断发出请求的是不是爬虫，是就拒绝访问服务。所以把`'User-Agent`换一下，假装发出的是浏览器的请求。

`'Referer'`是爬取视频内容必须的，没有就访问不了。

你可能会想问怎么知道请求头需要包含这些的。说来话长，慢慢摸出来的。不过我可以告诉你在哪获取。

<div align="center"><img src="headers.png" width="800" align="center" />

现在我们将请求头的构造封装一下。

In [58]:
def construct_headers(url):
    """Construct headers with the given url.

    :param url: Url needed to construct the headers.
    :return: Constructed headers.
    """

    return {
        'Referer': url,  # Necessary for requesting audio/video content.
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36',
    }

# 使用方法
url = construct_url(bv_id="BV19B4y1W76i")
headers = construct_headers(url=url)

然后改改前面4.4节定义的`request`方法。除了请求头，还顺便加上了超时限制`timeout`。请求后60秒不响应就中断请求。

In [59]:
def request(url, headers, timeout=60):
    """Send a request.

    :param url: The url to which the request is sent.
    :param headers: Request headers.
    :param timeout: Request timeout. Default 60s.
    :return: Response from the server of the url.
    """

    try:
        # Send the request.
        r = requests.get(
            url=url,
            headers=headers,
            timeout=timeout
        )

        # Raise an exception if something goes wrong.
        r.raise_for_status()

        return r
    except:
        return None

### 9.3 其他已知的信息

+ 由于视频为二进制数据，不能直接在html中存储，因此以**url**形式出现在html中。
+ 直接用浏览器点视频是拿不到视频url的，需要在浏览器的网页源码（或`requests`得到的`html`）中搜索。
+ 搜索关键字为："baseUrl"。可能会搜出来很多，不知道有什么区别。应该是差不多的。用第1个就可。
+ 注意，音频和视频的url是分开的。搜索"baseUrl"时，注意"**audio**"和"**video**"，两个都需要。
+ 获取audio和video的**url**的方式和第7节 “网页内容的差异”类似。
+ 获取audio和video的url后，再**请求其二进制内容**。通过`r.content`获取二进制内容。
+ 得到的音频和视频的二进制数据格式都为`m4s`。可以通过二进制文件写入（`"w"`改成`"wb"`）保存。
+ 保存后使用ffmpeg等工具将audio和video**合并**成mp4，就能得到视频内容。